# PLUMBER land surface model comparison project
The Protocol for the Analysis of Land Surface Models (PALS) Land Surface Model Benchmarking Evaluation Project (PLUMBER) was designed to be a land surface model (LSM) benchmarking intercomparison (Best et al., 2015). Thirteen Land Surface Models were compared across 20 different sites across the world (Figure and Table below from Best et al., 2015).

The PLUMBER experiment showed **that** models behave differently and created a motivation to find out **why** models behave differently. In this assignment you will use the Structure for Unifying Multiple Modelling Alternatives (Clark et al., 2015a, b) to investigate modelling decisions related to canopy processes and make a few steps towards answering the why-question.

<div>
<img src="./img/plumber_figure.png" width="750"/>
</div>

<div>
<img src="./img/plumber_table.png" width="750"/>
</div>

## References
Best, M.J., Abramowitz, G., Johnson, H.R., Pitman, A.J., Balsamo, G., Boone, A., Cuntz, M., Decharme, B., Dirmeyer, P.A., Dong, J. and Ek, M., 2015. The plumbing of land surface models: benchmarking model performance. Journal of Hydrometeorology, 16(3), pp.1425-1442.

Clark, M. P., Nijssen, B., Lundquist, J. D., Kavetski, D., Rupp, D. E., Woods, R. A., … Rasmussen, R. M. (2015a). A unified approach for process-based hydrologic modeling: 1. Modeling concept. Water Resources Research, 51(4), 2498–2514. https://doi.org/10.1002/2015WR017198

Clark, M. P., Nijssen, B., Lundquist, J. D., Kavetski, D., Rupp, D. E., Woods, R. A., … Marks, D. G. (2015b). A unified approach for process-based hydrologic modeling: 2. Model implementation and case studies. Water Resources Research, 51, 2515–2542. https://doi.org/10.1002/2015WR017200

Clark, M. P., Zolfaghari, R., Green, K. R., Trim, S., Knoben, W. J. M., Bennet, A., Nijssen, B., Ireson, A. and Spiteri, R. J. (under review). Laugh tests for land models. Submitted to Journal of Hydrometerology, 2020.

<div class="alert alert-block alert-danger">
<b>Caution:</b> Please make sure you are using the right Python virtual environment (aka Kernel) to run this Notebook.
</div>

## Loading the required HPC modules

Certain HPC modules need to be loaded beforehand. We are aiming to load the following list:
1. `openmpi/4.1.1-gnu`
2. `lib/hdf5/1.10.5-gnu730_openmpi402opagnu730`
3. `lib/netcdf/c-4.7.3_f-4.5.2_gnu730_hdf51105_openmpi402opagnu730`
4. `nco/4.9.2`
5. `gcc/7.3.0`
6. `python/3.10.4`

In [ ]:
import os

os.environ["LD_LIBRARY_PATH"] = (
    "/global/software/python/3.10.4/lib/:"
    "/global/software/gcc/gcc-7.3.0/lib64:"
    "/global/software/gcc/gcc-7.3.0/lib:"
    "/global/software/nco/nco492/lib:"
    "/global/software/netcdf/netcdfc-4.7.3_gnu730_hdf51105_openmpi402opagnu730/lib:"
    "/global/software/hdf5/hdf5-1.10.5_gnu730_openmpi402gnu730/lib:"
    "/global/software/openmpi/gnu-8.4.1/4.1.1/lib:"
    "/work/TALC/enci619.05_2024w/local/lib64/"
)

<br>

## Model setup
We'll first load the required modules.

In [ ]:
# modules 

import pysumma as ps
import xarray as xr
import matplotlib.pyplot as plt

<br>

Prof. Clark has created three PLUMBER-specific lookup tables for question 4 of this exercise. These bear the names `plumberCABLE`, `plumberCHTESSEL`, and `plumberSUMMA` respectively, Due to the experiment-specific nature of the `plumberCABLE`, `plumberCHTESSEL`, and `plumberSUMMA` lookup tables, we'll need to notify pysumma that these lookup tables are available. Run the code below to do this.

In [ ]:
# Update the Lookup table decision
ps.decisions.DECISION_META['vegeParTbl']['options'].append('plumberCABLE')
ps.decisions.DECISION_META['vegeParTbl']['options'].append('plumberCHTESSEL')
ps.decisions.DECISION_META['vegeParTbl']['options'].append('plumberSUMMA')

<br>

Now we can setup a pysumma simulation object. We'll abitrarily select the Amplero PLUMBER site as our default simulation.

In [ ]:
# SUMMA.exe location
executable = '../0_prerequisites/summa.exe'

In [ ]:
# Define location of .exe and file manager
file_manager_constant = './settings/plumber/Amplero/summa_zFileManager_Amplero.txt'

In [ ]:
# Open pysumma simulation object
s_amplero = ps.Simulation(executable, file_manager_constant)

In [ ]:
# Make the output directory if it doesn't exist
if not os.path.exists(s_amplero.manager['outputPath'].value):
    os.makedirs(s_amplero.manager['outputPath'].value)

<br>

## Assignment - subjective decisions in canopy processes
The image below is reproduced from Fig. 1 in Clark et al. (under review). The image shows the mass (blue) and energy (red) fluxes in the canopy and the canopy air space. The representation of these fluxes in process-based models is part of the many decisions a modeller makes. In this assigment you will investigate the impact of some of these decisions on the simulations for one or more PLUMBER sites.

<div>
<img src="./img/canopy_fluxes.png" width="750"/>
</div>

### Note on pysumma
In the following, you are asked to change certain values in your pysumma setup. Keep in mind that if you make these changes in the input files in the `settings` folder, you will need to make a new pysumma simulation object to have these changes take effect (`s_amplero = ps.Simulation(executable, file_manager_constant)`). If you instead make the changes to an existing pysumma simulation object, make sure to reset your pysumma simulation object to its base state after each simulation (`s_amplero.reset()`) so that your changes don't accumulate. 

### Note on finding simulated variables
In the following exercises, you are asked to investigate the impact of certain modelling options on model simulations.
Time series of certain simulated variables can be found in the output `.nc` file. You can find the location and name of the output `.nc` file by checking the `manager` option of your pysumma object (e.g. `print(s_amplero.manager)`). 

Interesting variables to investigate are indicated in each exercise. If you want to assess the impact on other simulations variables, you can either select those from the output `.nc` file that gets generated by default, or you can adapt the `Model_Output.txt` file in the `./settings/reynolds/` folder. Re-create your pysumma object afterwards to process changes to the `.txt`! See the SUMMA documentation for further guidance: https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/#output-control-file

<br>

<br>

## Exercise 1: Canopy transmissivity
The canopy can block incoming shortwave radiation (Qswd) from reaching the land surface. Transmission and attentuation of incoming shortwave radiation is controlled by the parametrization chosen to represent this process and the Leaf Area Index of the canopy. We'll focus on the parametrization choice for this assignment. This modelling decision is called `canopySrad` in the SUMMA decision file. 

Use your knowledge of pysumma to:

- Find out what the available parametrizations for `canopySrad` are
- Remove option `noah_mp` because it does not compute our variable of interest
- Run the simulations for your chosen PLUMBER site with the available parametrizations
- Make a graph to show the differences in simulated below canopy shortwave radiation (`scalarBelowCanopySolar`) and comment on the sensitivities. You can compare this with eastimated incoming shortwave radiation above the canopy for referece - this information is stored in the model forcing data as variable `SWRadAtm` but for convenience also generated as part of the SUMMA outputs for this experiment

As an example for the remainder of this exercise, code to run the required simulations for Exercise 1 is provided below. All that's left to complete this exercise is to run the code blocks below and to comment on the differences in snow depth simulation you see. 

#### Exercise 1: Answers

In [ ]:
# Find the parametrizations
decision = 'canopySrad'
parametrizations = s_amplero.decisions[decision].available_options.copy()
print(parametrizations)

In [ ]:
# The 'naoh_mp' parametrization option does not compute our variable of interest, `scalarBelowCanopySolar`, so there's no need to include it in the tests
parametrizations.remove('noah_mp')
print(parametrizations)

In [ ]:
# Do the simulations
for parametrization in parametrizations:
    
    # Print what we're doing
    print('Setting up simulations with parametrization ' + parametrization + ' for ' + decision)
    
    # reset the simulation
    s_amplero.reset()
    
    # Set the model decision
    s_amplero.decisions[decision] = parametrization
    
    # Run the model
    s_amplero.run('local', run_suffix=decision + '_' + parametrization )

In [ ]:
# prepare the plot
output_files = [
    './output/plumber/Amplero/Amplero__canopySrad_CLM_2stream_timestep.nc',
    './output/plumber/Amplero/Amplero__canopySrad_UEB_2stream_timestep.nc',
    './output/plumber/Amplero/Amplero__canopySrad_NL_scatter_timestep.nc',
    './output/plumber/Amplero/Amplero__canopySrad_BeersLaw_timestep.nc']

In [ ]:
# define a plotting function that collects a bunch of commands that need to be repeated to get each simulation into the plot
def aux_plot_1(files,labels):
    
    idx = -1 # label counter
    
    for file in files:
        
        # Increment counter
        idx +=1
        
        # load data
        ds = xr.open_dataset(file).isel(hru=0, gru=0).load() # pre-select the single hru and gru that are within the data
        
        # Plot the observations of first file
        if idx == 0:
            ds['SWRadAtm'].plot(color='k',label='Above canopy forcing')
        
        # Plot data 
        ds['scalarBelowCanopySolar'].plot(label=labels[idx])

In [ ]:
# make the plot
plt.figure(figsize=(30,6))
aux_plot_1(output_files,parametrizations)
plt.legend()
plt.title('Impact of shortwave transmissivity parametrization at the PLUMBER Amplero site')
plt.ylabel('Below canopy shortwave radiation [W m-2]');

<br>
<br>

## Exercise 2: Precipitation partitioning
Precipitation partitioning between canopy interception and throughfall is determined by both the canopy wetting function and the canopy interception capacity. The canopy wetting function is a modelling decision called `cIntercept`. SUMMA lists three options here, but only two are currently implemented. Canopy storage is controlled by parameters `refInterceptCapRain` and `refInterceptCapSnow`:

| Parameter                 | Value        | Min range    | Max range    | Unit | Description
| :-|:-|:-|:-|:-|:-
| refInterceptCapSnow       |       6.6000 |       1.0000 |      10.0000 | kg m-2 | canopy interception capacity per unit leaf area (snow)
| refInterceptCapRain       |       1.0000 |       0.0100 |       1.0000 | kg m-2 | canopy interception capacity per unit leaf area (rain)

Use your knowledge of pysumma to:
- Find out what the available parametrizations for `sIntercept` are
- Do not use parametrization `notPopulatedYet`, for obvious reasons
- Run the simulations for the Amplero site with the other two parametrizations
- Repeat this with several different values for `refInterceptCapRain` and `refInterceptCapSnow`
- Make graphs to show the differences in simulated evaporation (`scalarCanopyEvaporation`) and transpiration (`scalarCanopyTranspiration`), and the difference in simulated throughfall of rain (`scalarThroughfallRain`) and snow (`scalarThroughfallSnow`). Comment on what you see

<br>
<br>

## Exercise 3: Below canopy wind profile 
The choice of vertical wind profile affects the simulated sensible and latent heat fluxes. This modelling decision is called `windPrfile` in the SUMMA decision file. It has two options, `exponential` and `logBelowCanopy`, where the `exponential` profile also relies on a parameter that describes the decline in wind speed:

| Parameter                 | Value        | Min range    | Max range    | Unit | Description
| :-|:-|:-|:-|:-|:-
| windReductionParam        |       0.2800 |       0.0000 |       1.0000 | - | canopy wind reduction parameter

Use your knowledge of pysumma to:

- Run the simulations for your PLUMBER site with both profiles
- Re-run your simulations with the exponential wind profile several times with different values for the wind reduction parameter
- Make a graph to show the differences between the two wind profiles and the impact of the `windReductionParam` parameter. You can look at variables such as sensible heat (`scalarSenHeatTotal`) and surface temperature (`scalarSurfaceTemp`)

<br>
<br>

## Exercise 4: Vegetation properties
Canopy interception is partly determined by the specified Leaf Area Index (LAI) and Stem Area Index (SAI). SUMMA has two modelling options available as part of decision `LAI_method`: `monTable` and `specified`. `monTable` uses a lookup table to find LAI values, the choice of which is specified in the `vegeParTbl` modelling decision. Option `specified` instead uses parameter `summerLai` and `winterSAI` values from the global GRU parameter file. For now, we'll focus on the choice of lookup table only.

Prof. Clark has created three PLUMBER-specific lookup tables for this vegetation phenology experiment. These bear the names `plumberCABLE`, `plumberCHTESSEL`, and `plumberSUMMA` respectively, where the `plumberSUMMA` table is identical to the default PLUMBER setup we generated at the beginning of this assignment. 

For this exericse, use your knowledge of pysumma to:
- Generate simulations for one of the PLUMBER sites with each of the three phenology options specified above
- Make graphs to show the differences in simulated evaporation (`scalarCanopyEvaporation`) and transpiration (`scalarCanopyTranspiration`), and the difference in simulated throughfall of rain (`scalarThroughfallRain`) and snow (`scalarThroughfallSnow`). Comment on what you see

#### NOTE
Due to the experiment-specific nature of the `plumberCABLE`, `plumberCHTESSEL`, and `plumberSUMMA` lookup tables, some remapping of the `vegTypeIndex` parameter is needed. Detailed explanations are beyond the scope of this assignment. Use the code provided below to do the required remapping. To avoid confusion, your code must include:
- a line to change the `vegeParTbl` modelling decision to be one of `plumberCABLE`, `plumberCHTESSEL`, or `plumberSUMMA`
- a line that calls the remap function for your chosen site, e.g. `remap_vegTypeIndex_to_siteIndex(s_amplero,'Amplero')`

You can check if the remapping has been performed correctly by calling `s_amplero.local_attributes['vegTypeIndex'].data[0]` before you run your model simulation. The value returned must match the value in the code below, e.g. for the Amplero site you should see `1`, for Blodgett you should see `2`, etc.

Note that if you reset your simulation object (`s_amplero.reset()`), this will reset __both__ the modelling decision __and__ the remapping. 

In [ ]:
# remapping of the `vegTypeIndex` parameter
def remap_vegTypeIndex_to_siteIndex(simulationObject,site):
    
    # Indices of all sites in the `VEGPARM.TBL` and `MPTABLE.TBL` files, for the PLUMBER-specific lookup tables
    siteIndex = {'Amplero':  1,
                'Blodgett':  2,
                'Bugac':     3,
                'ElSaler':   4,
                'ElSaler2':  5,
                'Espirra':   6,
                'FortPeck':  7,
                'Harvard':   8,
                'Hesse':     9,
                'Howard':   10,
                'Howlandm': 11,
                'Hyytiala': 12,
                'Kruger':   13,
                'Loobos':   14,
                'Merbleue': 15,
                'Mopane':   16,
                'Palang':   17,
                'Sylvania': 18,
                'Tumba':    19,
                'UniMich':  20}
    
    # Change the `vegTypeIndex` parameter
    simulationObject.local_attributes['vegTypeIndex'].data[0] = siteIndex[site]
    
    return